In [ ]:
import pandas as pd
import numpy as np
import requests
import random
import folium
from folium.plugins import MarkerCluster
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
import geopandas as gpd
from shapely.geometry import Point
from sklearn.cluster import KMeans
import networkx as nx
from geopy.distance import geodesic
from haversine import haversine, Unit
import webbrowser
import chardet


# 한글 폰트 설정
plt.rcParams['font.family'] = 'Malgun Gothic'
plt.rcParams['axes.unicode_minus'] = False 

In [122]:
api_key = "af2e9120b2c913cfe45c5e181cb56f02"

In [ ]:
with open(r'C:/Users/leehy/OneDrive/바탕 화면/study/통계공모전/전처리데이터/park_Data_1.csv', 'rb') as f:
    result = chardet.detect(f.read(10000)) 
    print(result)

work_df = pd.read_csv(r"C:/Users/leehy/OneDrive/바탕 화면/study/통계공모전/data/한국장애인고용공단_장애인 표준사업장 현황_20241231.csv", encoding='cp949')
school_df = pd.read_csv(r"C:/Users/leehy/OneDrive/바탕 화면/study/통계공모전/data/서울시 특수학교 기본정보.csv", encoding='cp949')
b_station_df = pd.read_excel(r"C:/Users/leehy/OneDrive/바탕 화면/study/통계공모전/data/서울시버스정류소위치정보(20250513).xlsx")

bus_Sta = pd.read_csv(r'C:/Users/leehy/OneDrive/바탕 화면/study/통계공모전/전처리데이터/bus_sta_Data_1.csv')
park = pd.read_csv(r'C:/Users/leehy/OneDrive/바탕 화면/study/통계공모전/전처리데이터/park_Data_1.csv', encoding=result['encoding'])
work = pd.read_csv(r'C:/Users/leehy/OneDrive/바탕 화면/study/통계공모전/전처리데이터/work_Data_1.csv', encoding=result['encoding'])
school = pd.read_csv(r'C:/Users/leehy/OneDrive/바탕 화면/study/통계공모전/전처리데이터/school_Data_1.csv',encoding=result['encoding'])
organ= pd.read_csv(r'C:/Users/leehy/OneDrive/바탕 화면/study/통계공모전/전처리데이터/organ_Data_1.csv', encoding=result['encoding'])
hospital= pd.read_csv(r'C:/Users/leehy/OneDrive/바탕 화면/study/통계공모전/전처리데이터/hospital_Data_1.csv', encoding=result['encoding'])
library = pd.read_csv(r'C:/Users/leehy/OneDrive/바탕 화면/study/통계공모전/전처리데이터/library_Data_1.csv', encoding=result['encoding'])
large_apt = pd.read_csv(r'C:/Users/leehy/OneDrive/바탕 화면/study/통계공모전/전처리데이터/large_apt3_Data_1.csv', encoding=result['encoding'])
alpha = pd.read_csv(r'C:/Users/leehy/OneDrive/바탕 화면/study/통계공모전/전처리데이터/alpha_Data_1.csv', encoding=result['encoding'])


{'encoding': 'UTF-8-SIG', 'confidence': 1.0, 'language': ''}


In [25]:
# 각 데이터에 'source' 컬럼 추가
park['source'] = '공원'
work['source'] = '일자리'
school['source'] = '학교'
organ['source'] = '기관'
hospital['source'] = '병원'
library['source'] = '도서관'
large_apt['source'] = '아파트'

In [ ]:


def add_markers(map_obj, df, lat_col, lon_col, popup_col=None, color='blue', radius=5):
    for _, row in df.iterrows():
        lat = row[lat_col]
        lon = row[lon_col]

        if pd.notnull(lat) and pd.notnull(lon):
            folium.CircleMarker(
                location=[lat, lon],
                radius=radius,
                color=color,
                fill=True,
                fill_opacity=0.6,
                popup=row[popup_col] if popup_col and popup_col in row else None
            ).add_to(map_obj)           

In [27]:
# 중심점 설정 (서울 중심)
m = folium.Map(location=[37.5665, 126.9780], zoom_start=11)

#2. 공원
add_markers(m, park, lat_col='lat', lon_col='lon', popup_col='공원명', color='red')

#3.일자리
add_markers(m, work, lat_col='lat', lon_col='lon', popup_col='사업체명', color='orange')

#4. 학교
add_markers(m, school, lat_col='lat',lon_col='lon',popup_col='학교명',color='yellow')

#5. 장애인시설
add_markers(m, organ, lat_col='lat',lon_col='lon',popup_col='시설명',color='green')

#6. 병원
add_markers(m, hospital, lat_col='lat', lon_col='lon', popup_col='사업장명', color='blue')

#7. 도서관
add_markers(m, library, lat_col='lat', lon_col='lon', popup_col='도서관명', color='purple')

# 아파트
add_markers(m, large_apt, lat_col='lat', lon_col='lon', popup_col='아파트명', color='black')

m

In [28]:
m.save("ALL_map.html")

In [ ]:

webbrowser.open("ALL_map.html")

True

In [30]:
# 저상버스 정류소 위치 데이터 추출하기



low_bus_row = pd.read_excel(r"C:/Users/leehy/OneDrive/바탕 화면/study/통계공모전/data/서울시 저상버스 도입 노선 및 노선별 보유율(25.4.25).xlsx")
bus_id_row = pd.read_excel(r"C:/Users/leehy/OneDrive/바탕 화면/study/통계공모전/data/서울시버스노선ID정보(20250513).xlsx")
best_bus_station = pd.read_excel(r"C:/Users/leehy/OneDrive/바탕 화면/study/통계공모전/data/서울시버스노선별정류소정보(20250513).xlsx")
low_bus_row.rename(columns={'노선\n번호': '노선명', '인가\n대수': '인가 대수'}, inplace=True)

# 취약지역을 골라내기위해 저상버스 보급률이 70%이하인 노선명만 추출 -->총 123개의 노선추출
low_bus = low_bus_row[low_bus_row['저상보유율']<0.7]
#print(low_bus.to_string())

# 노선명 전처리 함수
def clean_route_name(series):
    return series.astype(str).str.strip().str.replace(r'\s+', '', regex=True)

# low_bus와 seoul_bus 모두 전처리
low_bus['노선명'] = clean_route_name(low_bus['노선명'])
bus_id_row['노선명'] = clean_route_name(bus_id_row['노선명'])

low_set = set(low_bus['노선명'])
seoul_set = set(bus_id_row['노선명'])

common = low_set & seoul_set
missing = low_set - seoul_set

print(f"공통 노선명 수: {len(common)}")     # 여기가 기대한 123이어야 함
print(f"일치하지 않는 노선명 수: {len(missing)}")
print(f"일치하지 않는 노선명 예시: {list(missing)[:10]}")

공통 노선명 수: 110
일치하지 않는 노선명 수: 13
일치하지 않는 노선명 예시: ['7013', '110', '8442', '서울04', '서울07', '8553', '서울10', '서울09', '서울05', '서울08']


C:\Users\leehy\AppData\Local\Temp\ipykernel_23392\3283267621.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  low_bus['노선명'] = clean_route_name(low_bus['노선명'])


# 가중치 조정 및 데이터 통합

In [31]:
hosp_w = hospital[hospital['업태구분명'] == '종합병원']
hosp_d = hospital[hospital['업태구분명'] == '요양병원(장애인의료재활시설)']

w_oversample= pd.concat([hosp_w]*2, ignore_index = True)
d_oversample = pd.concat([hosp_d] * 2 , ignore_index = True)

others = hospital[~hospital['업태구분명'].isin(['종합병원', '요양병원(장애인의료재활시설)'])]

hospital_w = pd.concat([others, w_oversample, d_oversample], ignore_index=True)

print("가중치 적용 전")
print(hospital['업태구분명'].value_counts())

print("\n가중치 적용 후")                         
print(hospital_w['업태구분명'].value_counts())
                         

가중치 적용 전
업태구분명
병원                 144
요양병원(일반요양병원)        75
한방병원                75
치과병원                47
종합병원                28
요양병원(정신병원)           1
요양병원(노인병원)           1
요양병원(장애인의료재활시설)      1
Name: count, dtype: int64

가중치 적용 후
업태구분명
병원                 144
요양병원(일반요양병원)        75
한방병원                75
종합병원                56
치과병원                47
요양병원(장애인의료재활시설)      2
요양병원(정신병원)           1
요양병원(노인병원)           1
Name: count, dtype: int64


In [33]:

def apply_weight(base_df, weight_df, radius=10):


    matches = []
    for idx, row in base_df.iterrows():
        base_point = (row['lat'], row['lon'])

        # weight_df의 좌표들과 거리 비교
        is_near = weight_df.apply(
            lambda x: haversine(base_point, (x['lat'], x['lon']), unit=Unit.METERS) <= radius,
            axis=1
        )
        if is_near.any():
            matches.append(row)

    if matches:
        oversampled = pd.DataFrame(matches)
        base_df = pd.concat([base_df, oversampled], ignore_index=True)

    return base_df

In [34]:
park_w = apply_weight(park, alpha, radius=30)
work_w = apply_weight(work, alpha, radius=20)
school_w = apply_weight(school, alpha, radius=30)
organ_w = apply_weight(organ, alpha, radius=20)
library_w= apply_weight(library, alpha, radius=20)
large_apt_w = apply_weight(large_apt, alpha, radius=70)

In [35]:
work_w[work_w.duplicated(keep = False)]

,사업체명,관할지사,소재지,정제주소,lat,lon,source
115,플라이쿱,서울북부지사,"서울특별시 성동구 동일로 287, 플라이쿱빌딩","서울특별시 성동구 동일로 287, 플라이쿱빌딩",37.557107,127.072096,일자리
134,플라이쿱,서울북부지사,"서울특별시 성동구 동일로 287, 플라이쿱빌딩","서울특별시 성동구 동일로 287, 플라이쿱빌딩",37.557107,127.072096,일자리


In [36]:

# 가중치 적용 후 병합
df_combined = pd.concat([
    park_w.loc[park.index.repeat(1)],
    work_w.loc[work.index.repeat(1)],
    school_w.loc[school.index.repeat(1)],
    organ_w.loc[organ.index.repeat(1)],
    hospital_w.loc[hospital.index.repeat(1)],
    library_w.loc[library.index.repeat(1)],
    large_apt_w.loc[large_apt.index.repeat(1),]
], ignore_index=True)  # 추후 가중치 수정가능

In [37]:

df_cleaned = df_combined.copy()

# 이름 컬럼 통일
df_cleaned['이름'] = (
    df_cleaned['공원명']
    .combine_first(df_cleaned['학교명'])
    .combine_first(df_cleaned['시설명'])
    .combine_first(df_cleaned['사업장명'])
    .combine_first(df_cleaned['도서관명'])
    .combine_first(df_cleaned['아파트명'])
)
6


df_cleaned['정제주소'] = df_cleaned['정제주소'].combine_first(df_cleaned['공원주소'])
combined= df_cleaned[['이름', '정제주소', 'lat', 'lon', 'source']]

시군구 추출

# 정류소 추출 알고리즘

## 기존 정류장 사용

In [ ]:


def connect_stops_and_facilities(bus_stops_df, facility_df, dist_threshold=250):
    G = nx.Graph()
    count = 0
    distance_records = []

    # 노드 추가 (정류장)
    for idx, row in bus_stops_df.iterrows():
        G.add_node(f"bus_{idx}", pos=(row['lat'], row['lon']), label='bus', name=row.get('name', f"bus_{idx}"))

    # 노드 추가 (시설)
    for idx, row in facility_df.iterrows():
        G.add_node(f"poi_{idx}", pos=(row['lat'], row['lon']), label=row['source'], name=row['이름'])

    # 엣지 추가 + 연결된 정보 저장
    for b_idx, b_row in bus_stops_df.iterrows():
        b_coord = (b_row['lat'], b_row['lon'])
        for p_idx, p_row in facility_df.iterrows():
            p_coord = (p_row['lat'], p_row['lon'])
            dist = geodesic(b_coord, p_coord).meters
            if dist <= dist_threshold:
                G.add_edge(f"bus_{b_idx}", f"poi_{p_idx}", weight=dist)
                count += 1
                distance_records.append({
                    '정류장_id': f"bus_{b_idx}",
                    'name': b_row.get('name', f"bus_{b_idx}"),
                    '정류장_lat': b_row['lat'],
                    '정류장_lon': b_row['lon'],
                    '시설_id': f"poi_{p_idx}",
                    '시설_이름': p_row['이름'],
                    '시설_source': p_row['source'],
                    '시설_lat': p_row['lat'],
                    '시설_lon': p_row['lon'],
                    '거리(m)': dist
                })

    print(f"✅ 연결된 정류장-시설 페어 수: {count}")

    # 데이터프레임 정리
    distance_df = pd.DataFrame(distance_records)

    # 정류장별 연결된 시설 수 계산
    stop_facility_count = distance_df.groupby('name').size().reset_index(name='연결된_시설_수')

    # 좌표 병합
    stop_coords = bus_stops_df[['name', 'lat', 'lon']]
    stop_facility_count = pd.merge(stop_facility_count, stop_coords, on='name', how='left')

    # 지도 시각화
    center = [bus_stops_df['lat'].mean(), bus_stops_df['lon'].mean()]
    m = folium.Map(location=center, zoom_start=13)

    # 정류장 시각화
    for idx, row in bus_stops_df.iterrows():
        folium.CircleMarker(
            location=[row['lat'], row['lon']],
            radius=4, color='blue', fill=True, fill_color='blue',
            popup=f"정류장: {row.get('name', idx)}"
        ).add_to(m)

    # 시설 시각화
    color_map = {'공원': 'green', '일자리': 'orange', '기관': 'purple', '학교': 'red', '도서관': 'yellow'}
    for idx, row in facility_df.iterrows():
        folium.CircleMarker(
            location=[row['lat'], row['lon']],
            radius=4,
            color=color_map.get(row['source'], 'gray'),
            fill=True,
            fill_color=color_map.get(row['source'], 'gray'),
            popup=f"{row['source']}: {row['이름']}"
        ).add_to(m)

    # 간선 시각화
    for u, v, d in G.edges(data=True):
        u_latlon = G.nodes[u]['pos']
        v_latlon = G.nodes[v]['pos']
        folium.PolyLine(
            locations=[u_latlon, v_latlon],
            color='gray', weight=1
        ).add_to(m)

    return distance_df, stop_facility_count, m

In [52]:
def deduplicate_stops_by_name(stop_count_df: pd.DataFrame) -> pd.DataFrame:
    
    required_columns = {'name', '연결된_시설_수', 'lat', 'lon'}
    missing = required_columns - set(stop_count_df.columns)
    if missing:
        raise ValueError(f"필수 컬럼이 누락되었습니다: {missing}")

    deduped = (
        stop_count_df
        .groupby('name')
        .agg({
            '연결된_시설_수': 'max',  # 또는 'mean'
            'lat': 'mean',
            'lon': 'mean'
        })
        .reset_index()
    )
    return deduped


In [39]:
def plot_stop_facility_counts(stop_facility_df, center=None, major_threshold=3):

    if center is None:
        center = [stop_facility_df['lat'].mean(), stop_facility_df['lon'].mean()]

    major_stops = stop_facility_df[stop_facility_df['연결된_시설_수'] >= major_threshold]
    minor_stops = stop_facility_df[stop_facility_df['연결된_시설_수'] < major_threshold]

    m = folium.Map(location=center, zoom_start=13)

    # Major stops (3개 이상)
    for _, row in major_stops.iterrows():
        folium.CircleMarker(
            location=[row['lat'], row['lon']],
            radius=6, color='blue', fill=True, fill_color='blue',
            popup=f"[{major_threshold}개 이상] {row['name']} ({row['연결된_시설_수']}개)"
        ).add_to(m)

    # Minor stops (2개 이하)
    for _, row in minor_stops.iterrows():
        folium.CircleMarker(
            location=[row['lat'], row['lon']],
            radius=6, color='red', fill=True, fill_color='red',
            popup=f"[{major_threshold - 1}개 이하] {row['name']} ({row['연결된_시설_수']}개)"
        ).add_to(m)

    return m, major_stops, minor_stops


## 신설 정류장 설정

In [ ]:


def get_uncovered_facilities(major_stops: pd.DataFrame, facilities_df: pd.DataFrame, buffer_m: int = 250) -> pd.DataFrame:

    gdf_stops = gpd.GeoDataFrame(
        major_stops.copy(),
        geometry=gpd.points_from_xy(major_stops['lon'], major_stops['lat']),
        crs='EPSG:4326'
    ).to_crs(epsg=5179)

    gdf_stops['buffer'] = gdf_stops.geometry.buffer(buffer_m)
    gdf_buffer = gdf_stops.set_geometry('buffer').to_crs(epsg=4326)

    gdf_facilities = gpd.GeoDataFrame(
        facilities_df.copy(),
        geometry=gpd.points_from_xy(facilities_df['lon'], facilities_df['lat']),
        crs='EPSG:4326'
    )
    joined = gpd.sjoin(gdf_facilities, gdf_buffer, how='left', predicate='within')
    uncovered = joined[joined['index_right'].isna()].copy()

    uncovered['lat'] = uncovered.geometry.y
    uncovered['lon'] = uncovered.geometry.x

    return uncovered[['lat', 'lon'] + [col for col in uncovered.columns if col not in ['lat', 'lon', 'geometry']]]  # 기타 메타데이터 유지


In [ ]:


def plot_uncovered_facilities(uncovered: pd.DataFrame, center_lat=37.5665, center_lon=126.9780, zoom=12) -> folium.Map:
    m = folium.Map(location=[center_lat, center_lon], zoom_start=zoom)
    for _, row in uncovered.iterrows():
        folium.CircleMarker(
            location=[row['lat'], row['lon']],
            radius=5,
            color='red',
            fill=True,
            fill_color='red',
            fill_opacity=0.7,
            popup=row.get('시설명', '접근 불가 시설')
        ).add_to(m)
    return m


In [ ]:


def cluster_uncovered_facilities(uncovered: pd.DataFrame, n_clusters: int = 8) -> tuple[pd.DataFrame, folium.Map]:

    coords = uncovered[['lat', 'lon']]
    kmeans = KMeans(n_clusters=n_clusters, random_state=42)
    uncovered = uncovered.copy()
    uncovered['cluster'] = kmeans.fit_predict(coords)

    # 클러스터 중심 좌표
    centroids = kmeans.cluster_centers_
    centroid_df = pd.DataFrame(centroids, columns=['lat', 'lon'])
    centroid_df['cluster'] = centroid_df.index

    # 지도
    center_lat, center_lon = uncovered['lat'].mean(), uncovered['lon'].mean()
    m = folium.Map(location=[center_lat, center_lon], zoom_start=14)
    colors = ['red', 'blue', 'green', 'purple', 'orange', 'darkgreen', 'cadetblue', 'black']

    for _, row in uncovered.iterrows():
        color = colors[row['cluster'] % len(colors)]
        folium.CircleMarker(
            location=[row['lat'], row['lon']],
            radius=5,
            color=color,
            fill=True,
            fill_color=color,
            fill_opacity=0.8,
            popup=f"Cluster: {row['cluster']}, 시설명: {row.get('시설명', '-')}"
        ).add_to(m)
        folium.PolyLine(
            locations=[[row['lat'], row['lon']], centroids[row['cluster']]],
            color=color,
            weight=2,
            opacity=0.5
        ).add_to(m)

    # 중심점 추가
    for _, row in centroid_df.iterrows():
        folium.CircleMarker(
            location=[row['lat'], row['lon']],
            radius=9,
            color='gold',
            fill=True,
            fill_color='gold',
            fill_opacity=1.0,
            popup=f"Cluster {row['cluster']} 중심"
        ).add_to(m)

    # 클러스터 중심지 → new 정류장 후보
    new_stops = centroid_df[['lat', 'lon']].copy()
    new_stops['name'] = [f'new_{i+1}' for i in range(len(new_stops))]

    return new_stops, m


In [ ]:


def plot_final_stops(final_stops: pd.DataFrame, zoom_start: int = 13) -> folium.Map:

    center = [final_stops['lat'].mean(), final_stops['lon'].mean()]
    m = folium.Map(center, zoom_start=zoom_start)

    for _, row in final_stops.iterrows():
        color = 'yellow' if str(row['name']).startswith('new') else 'green'
        folium.CircleMarker(
            location=[row['lat'], row['lon']],
            radius=6,
            color=color,
            fill=True,
            fill_color=color,
            fill_opacity=0.9,
            popup=row['name']
        ).add_to(m)

    return m


# 경로설정알고리즘

In [83]:
def get_distance(p1, p2):
    """
    Return straight-line distance (meters) between p1 and p2 using haversine.
    """
    return haversine(p1, p2) * 1000

In [84]:
def compute_total_distance(path, coords):
    """Compute total round-trip straight-line distance for given path indices."""
    dist = 0
    for i in range(len(path) - 1):
        dist += get_distance(coords[path[i]], coords[path[i+1]])
    dist += get_distance(coords[path[-1]], coords[path[0]])
    return dist

In [85]:
def greedy_path(coords):
    print("Running Greedy Algorithm...")
    n = len(coords)
    visited = [0]
    unvisited = set(range(1, n))
    while unvisited:
        last = visited[-1]
        next_city = min(unvisited, key=lambda i: get_distance(coords[last], coords[i]))
        visited.append(next_city)
        unvisited.remove(next_city)
    print(f"Greedy final distance: {compute_total_distance(visited, coords):.0f}m")
    return visited

In [86]:
def two_opt(path, coords):
    print("Running 2-opt improvement...")
    best = path[:]
    best_dist = compute_total_distance(best, coords)
    improved = True
    while improved:
        improved = False
        for i in range(1, len(best) - 2):
            for j in range(i+1, len(best)):
                if j - i == 1:
                    continue
                new_path = best[:i] + best[i:j][::-1] + best[j:]
                new_dist = compute_total_distance(new_path, coords)
                if new_dist < best_dist:
                    best, best_dist, improved = new_path, new_dist, True
                    print(f"Improved to distance: {best_dist:.0f}m")
        if not improved:
            print("No more improvements found.")
    print(f"2-opt final distance: {best_dist:.0f}m")
    return best

In [87]:

def simulated_annealing(coords, T=10000, cooling_rate=0.995):
    print("Running Simulated Annealing...")
    n = len(coords)
    current = list(range(n))
    random.shuffle(current)
    best = current[:]
    current_dist = best_dist = compute_total_distance(current, coords)
    while T > 1:
        i, j = sorted(random.sample(range(n), 2))
        new = current[:]
        new[i:j] = reversed(new[i:j])
        new_dist = compute_total_distance(new, coords)
        if new_dist < current_dist or random.random() < np.exp((current_dist - new_dist) / T):
            current, current_dist = new, new_dist
            if new_dist < best_dist:
                best, best_dist = new, new_dist
        T *= cooling_rate
    print(f"Simulated Annealing final distance: {best_dist:.0f}m")
    return best


In [ ]:

def genetic_algorithm(coords, pop_size=50, generations=100, mutation_rate=0.1, elite_size=5):
    print("Running Genetic Algorithm...")
    n = len(coords)
    def crossover(p1, p2):
        cut = random.randint(1, n-2)
        return p1[:cut] + [x for x in p2 if x not in p1[:cut]]
    def mutate(p):
        i, j = sorted(random.sample(range(n), 2))
        p[i], p[j] = p[j], p[i]
    population = [random.sample(range(n), n) for _ in range(pop_size)]
    for gen in range(generations):
        population.sort(key=lambda p: compute_total_distance(p, coords))
        new_pop = population[:elite_size]
        while len(new_pop) < pop_size:
            p1, p2 = random.sample(population[:elite_size], 2)
            child = crossover(p1, p2)
            if random.random() < mutation_rate:
                mutate(child)
            new_pop.append(child)
        population = new_pop
        print(f"Gen {gen} best distance: {compute_total_distance(population[0], coords):.0f}m")
    best = population[0]
    print(f"Genetic final distance: {compute_total_distance(best, coords):.0f}m")
    return best

In [89]:
def run_all_algorithms(df):
    """
    Runs all TSP algorithms on df, plots all on m_all, best on m_best.
    """
    coords = list(zip(df['lat'], df['lon']))
    names = df['name'].tolist()

    # 1. Compute paths
    path_greedy = greedy_path(coords)
    path_2opt = two_opt(path_greedy, coords)
    path_simann = simulated_annealing(coords)
    path_genetic = genetic_algorithm(coords)

    results = {
        'Greedy': path_greedy,
        '2-opt': path_2opt,
        'SimAnn': path_simann,
        'Genetic': path_genetic
    }

    # 2. Compute distances
    distances = {k: compute_total_distance(v, coords) for k, v in results.items()}

    # 3. Plot all routes
    center = [df['lat'].mean(), df['lon'].mean()]
    m_all = folium.Map(location=center, zoom_start=13)
    colors = {'Greedy':'blue', '2-opt':'red', 'SimAnn':'purple', 'Genetic':'orange'}

    # Mark stops
    for _, r in df.iterrows():
        c = 'yellow' if r['name'].startswith('new') else 'green'
        folium.CircleMarker([r.lat, r.lon], radius=5, color=c, fill=True, fill_color=c).add_to(m_all)

    # Draw straight lines
    for k, path in results.items():
        latlons = [coords[i] for i in path] + [coords[path[0]]]
        folium.PolyLine(latlons, color=colors[k], weight=4, opacity=0.7, tooltip=f"{k}: {distances[k]:.0f}m").add_to(m_all)

    # 4. Identify best
    best_algo = min(distances, key=distances.get)
    best_path = results[best_algo]
    print(f"Best algorithm: {best_algo} with distance {distances[best_algo]:.0f}m")

    # 5. Plot best only
    m_best = folium.Map(location=center, zoom_start=13)
    for _, r in df.iterrows():
        c = 'yellow' if r['name'].startswith('new') else 'green'
        folium.CircleMarker([r.lat, r.lon], radius=5, color=c, fill=True, fill_color=c).add_to(m_best)
    latlons = [coords[i] for i in best_path] + [coords[best_path[0]]]
    folium.PolyLine(latlons, color='black', weight=5, opacity=0.9).add_to(m_best)

    return m_all, m_best, results, best_algo


In [90]:
#실제 도로에 적용
def draw_best_path_on_road(df, results, best_algo, api_key):
    coords = list(zip(df['lat'], df['lon']))
    path = results[best_algo]
    
    m_road = folium.Map(location=[df['lat'].mean(), df['lon'].mean()], zoom_start=13)
    
    # 정류장 마커
    for _, r in df.iterrows():
        c = 'yellow' if str(r['name']).startswith('new') else 'green'
        folium.CircleMarker([r.lat, r.lon], radius=5, color=c, fill=True, fill_color=c).add_to(m_road)
    
    # 실제 도로 경로 요청 및 그리기
    for i in range(len(path)):
        p1 = coords[path[i]]
        p2 = coords[path[(i + 1) % len(path)]]  # 순환 경로
        try:
            url = "https://apis-navi.kakaomobility.com/v1/directions"
            headers = {"Authorization": f"KakaoAK {api_key}"}
            params = {
                "origin": f"{p1[1]},{p1[0]}",  # lon,lat
                "destination": f"{p2[1]},{p2[0]}",
                "priority": "RECOMMEND",
            }
            resp = requests.get(url, headers=headers, params=params)
            data = resp.json()
            steps = data['routes'][0]['sections'][0]['roads']
            polyline = []
            for road in steps:
                verts = road['vertexes']
                for i in range(0, len(verts), 2):
                    lat = verts[i+1]
                    lon = verts[i]
                    polyline.append((lat, lon))
            folium.PolyLine(polyline, color='black', weight=5, opacity=0.9).add_to(m_road)
        except Exception as e:
            print(f"[ROAD ERROR] from {p1} to {p2}: {e}")
    
    return m_road


In [91]:
bus_Sta.rename(columns={
    'X좌표': 'lon',
    'Y좌표': 'lat',
    '정류소명': 'name'
}, inplace=True)

# 강북구 적용

In [ ]:
# 강북구 

# 버스정류장 -> GeoDataFrame
bus_Sta['geometry'] = bus_Sta.apply(lambda row: Point(row['lon'], row['lat']), axis=1)
gdf = gpd.GeoDataFrame(bus_Sta, geometry='geometry', crs='EPSG:4326')  # WGS84 좌표계

# 서울시 행정구역 데이터 로드 (예: shp 파일 필요)
seoul_gu = gpd.read_file(r'C:/Users/leehy/OneDrive/바탕 화면/study/통계공모전/전처리데이터/LARD_ADM_SECT_SGG_11_202505.shp')  # 또는 서울시 행정경계 geojson 등
seoul_gu = seoul_gu.to_crs('EPSG:4326')  # 좌표계 일치

# 강북구만 필터
gangbuk = seoul_gu[seoul_gu['SGG_NM'] == '서울특별시 강북구']

# 정류장이 강북구에 속하는지 판단
gangbuk_stops = gdf[gdf.within(gangbuk.geometry.values[0])]

In [43]:
gangbuk_df = combined[combined['정제주소'].str.contains('강북구', na=False)]

In [57]:
distance_df, stop_count_df, result_map = connect_stops_and_facilities(
    bus_stops_df=gangbuk_stops,
    facility_df=gangbuk_df,
    dist_threshold=250
)

display(distance_df.head())
display(stop_count_df.sort_values(by='연결된_시설_수', ascending=False).head())
result_map

✅ 연결된 정류장-시설 페어 수: 176


,정류장_id,name,정류장_lat,정류장_lon,시설_id,시설_이름,시설_source,시설_lat,시설_lon,거리(m)
0,bus_3024,수유3동우체국,37.641346,127.029161,poi_309,강북구수어통역센터,기관,37.641356,127.029629,41.287438
1,bus_3024,수유3동우체국,37.641346,127.029161,poi_400,서울현대병원,병원,37.640256,127.028504,134.138365
2,bus_3025,수유3동우체국,37.640893,127.028827,poi_309,강북구수어통역센터,기관,37.641356,127.029629,87.495672
3,bus_3025,수유3동우체국,37.640893,127.028827,poi_400,서울현대병원,병원,37.640256,127.028504,76.231821
4,bus_3025,수유3동우체국,37.640893,127.028827,poi_689,경희치과병원,병원,37.639136,127.027475,228.639713


,name,연결된_시설_수,lat,lon
109,수유3동우체국,11,37.640060,127.028116
108,수유3동우체국,11,37.640026,127.027579
106,수유3동우체국,11,37.641346,127.029161
107,수유3동우체국,11,37.640893,127.028827
55,미아역.신일중고,7,37.624295,127.027045


In [ ]:
deduped_stops = deduplicate_stops_by_name(stop_count_df)
deduped_stops.sort_values(by='연결된_시설_수', ascending=False)


,name,연결된_시설_수,lat,lon
59,수유3동우체국,11,37.640581,127.028421
26,미아역.신일중고,7,37.624582,127.026847
5,강북구청,6,37.640469,127.026070
6,강북구청사거리,6,37.638865,127.028479
10,광산사거리,6,37.643226,127.022600
...,...,...,...,...
44,삼양동벽산라이브파크미양초등학교,1,37.620308,127.012607
46,삼양시장,1,37.625232,127.017352
47,삼양신협,1,37.625528,127.023406
52,서울정인학교,1,37.629213,127.013002


In [60]:
deduped_stops['연결된_시설_수'].value_counts()

연결된_시설_수
1     35
2     33
4      9
6      3
7      1
11     1
3      1
Name: count, dtype: int64

In [62]:
m, major_stops, minor_stops = plot_stop_facility_counts(deduped_stops)


In [63]:
m

In [64]:
major_stops

,name,연결된_시설_수,lat,lon
4,강북구민운동장.4단지,4,37.630073,127.038553
5,강북구청,6,37.640469,127.026070
6,강북구청사거리,6,37.638865,127.028479
10,광산사거리,6,37.643226,127.022600
26,미아역.신일중고,7,37.624582,127.026847
32,번동슈퍼,4,37.627683,127.037723
33,번동주공아파트3단지4단지,4,37.629745,127.040537
35,번동초등학교,4,37.628735,127.039588
36,번동한솔솔파크,4,37.627599,127.038047
59,수유3동우체국,11,37.640581,127.028421


In [71]:
uncovered = get_uncovered_facilities(major_stops, gangbuk_df)


plot_uncovered_facilities(uncovered)

new_stops, cluster_map = cluster_uncovered_facilities(uncovered, n_clusters=8)

# 결과 확인
display(new_stops)
cluster_map

c:\Users\leehy\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1446: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(


,lat,lon,name
0,37.615491,127.031676,new_1
1,37.637196,127.013007,new_2
2,37.620108,127.016400,new_3
3,37.635470,127.022673,new_4
4,37.624549,127.000938,new_5
5,37.652331,127.011624,new_6
6,37.623336,127.040734,new_7
7,37.632796,127.031727,new_8


In [ ]:

combined_stops = pd.concat([major_stops, new_stops], ignore_index=True)
final_stops = combined_stops[['name','lat','lon']]
final_stops

,name,lat,lon
0,강북구민운동장.4단지,37.630073,127.038553
1,강북구청,37.640469,127.026070
2,강북구청사거리,37.638865,127.028479
3,광산사거리,37.643226,127.022600
4,미아역.신일중고,37.624582,127.026847
5,번동슈퍼,37.627683,127.037723
6,번동주공아파트3단지4단지,37.629745,127.040537
7,번동초등학교,37.628735,127.039588
8,번동한솔솔파크,37.627599,127.038047
9,수유3동우체국,37.640581,127.028421


In [82]:
m = plot_final_stops(final_stops)
m

# 금천구 적용

In [92]:
geumcheon = seoul_gu[seoul_gu['SGG_NM'] == '서울특별시 금천구']

# 정류장이 강북구에 속하는지 판단
geumcheon_stops = gdf[gdf.within(geumcheon .geometry.values[0])]

geumcheon_df = combined[combined['정제주소'].str.contains('금천구', na=False)]

In [104]:
distance_df, stop_count_df, result_map = connect_stops_and_facilities(
    bus_stops_df=geumcheon_stops,
    facility_df=geumcheon_df,
    dist_threshold=250
)

display(distance_df.head())
display(stop_count_df.sort_values(by='연결된_시설_수', ascending=False).head())
result_map

✅ 연결된 정류장-시설 페어 수: 300


,정류장_id,name,정류장_lat,정류장_lon,시설_id,시설_이름,시설_source,시설_lat,시설_lon,거리(m)
0,bus_6915,한일유앤아이아파트,37.485646,126.876035,poi_154,NaN,일자리,37.484967,126.874545,151.841594
1,bus_7146,한일유엔아이APT,37.485618,126.875950,poi_154,NaN,일자리,37.484967,126.874545,143.811241
2,bus_7351,문성초등학교,37.474034,126.898330,poi_532,새움병원,병원,37.473053,126.897655,124.168825
3,bus_7352,금천우체국,37.468978,126.898030,poi_533,서울바른세상병원,병원,37.471162,126.897741,243.706541
4,bus_7353,금천우체국,37.470136,126.898003,poi_533,서울바른세상병원,병원,37.471162,126.897741,116.203874


,name,연결된_시설_수,lat,lon
16,가산디지털단지역,16,37.480782,126.884209
10,가산디지털단지역,16,37.481293,126.881042
18,가산디지털단지역,16,37.481289,126.883113
17,가산디지털단지역,16,37.481212,126.882968
15,가산디지털단지역,16,37.480205,126.883310


In [105]:
deduped_stops = deduplicate_stops_by_name(stop_count_df)
deduped_stops.sort_values(by='연결된_시설_수', ascending=False)

,name,연결된_시설_수,lat,lon
8,가산디지털단지역,16,37.481050,126.882795
5,STXV타워,12,37.476912,126.883199
63,벽산디지털밸리,12,37.476474,126.885179
43,독산역,11,37.466373,126.888851
49,디지털3단지사거리,8,37.476098,126.880789
...,...,...,...,...
61,벽산1단지입구,1,37.450143,126.919849
62,벽산6단지입구,1,37.449960,126.918117
65,빅마켓,1,37.470030,126.895412
70,성지아트빌,1,37.473606,126.889197


In [116]:
m, major_stops, minor_stops = plot_stop_facility_counts(deduped_stops, major_threshold=5)

In [117]:
m

In [118]:
major_stops.head()

,name,연결된_시설_수,lat,lon
1,LG전자,7,37.470657,126.885886
3,RSM타워.르노코리아서부사업소,5,37.471015,126.883112
5,STXV타워,12,37.476912,126.883199
8,가산디지털단지역,16,37.481050,126.882795
15,교학사,6,37.469241,126.886211


In [119]:
uncovered = get_uncovered_facilities(major_stops, geumcheon_df)


plot_uncovered_facilities(uncovered)

new_stops, cluster_map = cluster_uncovered_facilities(uncovered, n_clusters=5)

# 결과 확인
display(new_stops)
cluster_map

c:\Users\leehy\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1446: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(


,lat,lon,name
0,37.469497,126.894898,new_1
1,37.448274,126.918767,new_2
2,37.472812,126.908017,new_3
3,37.484967,126.874545,new_4
4,37.446866,126.904013,new_5


In [120]:

combined_stops = pd.concat([major_stops, new_stops], ignore_index=True)
final_stops = combined_stops[['name','lat','lon']]
final_stops

,name,lat,lon
0,LG전자,37.470657,126.885886
1,RSM타워.르노코리아서부사업소,37.471015,126.883112
2,STXV타워,37.476912,126.883199
3,가산디지털단지역,37.481050,126.882795
4,교학사,37.469241,126.886211
5,금천구종합청사.금천구청역,37.456208,126.895116
6,대륭테크노2차.12차,37.468830,126.884797
7,대륭테크노타운,37.469664,126.883559
8,대륭테크노타운17차,37.468514,126.886643
9,독산역,37.466373,126.888851


In [121]:
m = plot_final_stops(final_stops)
m

In [ ]:
m_all, m_best, results, best_algo = run_all_algorithms(final_stops)
m_all  # 전체 경로
m_best # 최적 경로

Running Greedy Algorithm...
Greedy final distance: 16501m
Running 2-opt improvement...
Improved to distance: 16193m
Improved to distance: 16182m
Improved to distance: 16061m
Improved to distance: 16025m
Improved to distance: 15892m
Improved to distance: 15680m
Improved to distance: 15553m
Improved to distance: 15544m
No more improvements found.
2-opt final distance: 15544m
Running Simulated Annealing...
Simulated Annealing final distance: 13996m
Running Genetic Algorithm...
Gen 0 best distance: 33936m
Gen 1 best distance: 29712m
Gen 2 best distance: 28793m
Gen 3 best distance: 28054m
Gen 4 best distance: 28054m
Gen 5 best distance: 28054m
Gen 6 best distance: 26783m
Gen 7 best distance: 26783m
Gen 8 best distance: 25315m
Gen 9 best distance: 24879m
Gen 10 best distance: 24812m
Gen 11 best distance: 24714m
Gen 12 best distance: 24714m
Gen 13 best distance: 23919m
Gen 14 best distance: 21046m
Gen 15 best distance: 21034m
Gen 16 best distance: 21034m
Gen 17 best distance: 21034m
Gen 18 be

In [129]:
m_all

In [128]:
m_road = draw_best_path_on_road(final_stops, results, best_algo, api_key=api_key)
m_road  